# An implementation of gradient descent to optimize shallow neural network's parameters

In [1]:
import numpy as np

# Helper functions

### Activation functions

In [8]:
# ReLu
def ReLu(Z):
    """
    This function applies a ReLu transformation to the input Z.
    """
    
    return np.maximum(0, Z)


# test
Z_test = np.array([[-1, 2, 3, -4], [1, 5, -3, 2]])
print("ReLu(Z_test) =", ReLu(Z_test))

ReLu(Z_test) = [[0 2 3 0]
 [1 5 0 2]]


In [9]:
# sigmoid
def sigmoid(Z):
    """
    This function applies a Sigmoid transformation to the input Z.
    """
    
    return 1 / (1 + np.exp(Z))


# test
Z_test = np.array([[-1, 2, 3, -4], [1, 5, -3, 2]])
print("sigmoid(Z_test) =", sigmoid(Z_test))

sigmoid(Z_test) = [[0.73105858 0.11920292 0.04742587 0.98201379]
 [0.26894142 0.00669285 0.95257413 0.11920292]]


### Forward propagation

### <span style="color:red">TODO</span>

In [ ]:
# one layer of forward prop
def get_activations(A_input, W, b, activation):
    """
    This function computes and returns the activisions of one layer of neurons.
    """
    
    Z = np.dot(W, A_input) + b
    return activision(Z)


# test
A_input_test = np.array([[-1, 2, 3, -4], [1, 5, -3, 2]])
W_test = np.array([[1, 2], [0, -1], []])